In [1]:
import numpy as np
import scipy
from scipy import signal
import mne
import glob
from sklearn.decomposition import PCA
import csv

In [2]:
local_path = './../../../data/raw/HV/HV1/HV1_F1_L_Removed_ICA.set'


In [3]:
def bandpower(x, fs, fmin, fmax):
    f, Pxx = signal.periodogram(x, fs=fs)
    ind_min = scipy.argmax(f > fmin) - 1
    ind_max = scipy.argmax(f > fmax) - 1
    return scipy.trapz(Pxx[ind_min: ind_max], f[ind_min: ind_max])

In [4]:
data = mne.io.read_epochs_eeglab(local_path)._data

Extracting parameters from ./../../../data/raw/HV/HV1/HV1_F1_L_Removed_ICA.set...


<ipython-input-4-e0a8228b57c9>:1: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  data = mne.io.read_epochs_eeglab(local_path)._data


56 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.


In [5]:
data.shape

(56, 61, 1250)

In [6]:
test_bp = bandpower(data[0][0], 250, 13, 30)

In [7]:
test_bp

1.7894988777039825e-12

In [8]:
def pca(channels, n_components=5):
    pca = PCA(n_components=n_components)
    return pca.fit_transform(channels)

#### This will be used for logging

In [9]:
def log_result(file, name, accuracy, patients_correct, patients_total, set_name, channels, notes):
    writer = csv.writer(file)
    writer.writerow([name, str(accuracy*100) + '%', ' ' + str(patients_correct) + '/' + str(patients_total), set_name, channels, notes])
        

# Apply bandpower to datasets

In [10]:
# Returns relevant datasets (f.e. all right-hand recordings of healthy patients) as a list of np arrays
def get_datasets(patient_type_location, recording_type_expression):
    if recording_type_expression != l_new:
        sets_locations = glob.glob(patient_type_location + recording_type_expression + suffix)
    else:
        sets_locations = glob.glob(patient_type_location + recording_type_expression)
    
    sets = []
    for path in sets_locations: 
        sets.append(mne.io.read_epochs_eeglab(path))
        
    return np.array(np.array([(patient._data) for patient in sets]))

In [11]:
# Calculate bandpower for all channels for a patient
def channels_bandpower(channels):
    fs = 250
    b = bandpower
    return np.array(list(map( lambda arr: [b(arr, fs, 4, 8), b(arr, fs, 8, 13), b(arr, fs, 13, 30)], channels)))

In [12]:
root = './../../../'
suffix = '*.set'

location_healthy = root + 'data/raw/HV/*/'
location_pain = root + 'data/raw/PP/*/'
location_nopain = root + 'data/raw/PnP/*/'

location_pwp = root + 'data_new/raw/PwP/*/'
location_pdp = root + 'data_new/raw/PdP/*/'
location_pnp = root + 'data_new/raw/PnP/*/'


rh = '*_RH*'
lh = '*_LH*'
l_new = '*_L.set'   # NO SUFFIX
l_old = '*_L_*'

sets_healthy_rh = glob.glob(location_pnp + l_new)

sets_healthy_rh

['./../../../data_new/raw/PnP\\PnP_1\\PnP_1_L.set',
 './../../../data_new/raw/PnP\\PnP_3\\PnP_3_L.set',
 './../../../data_new/raw/PnP\\PnP_4\\PnP_4_L.set',
 './../../../data_new/raw/PnP\\PnP_5\\PnP_5_L.set',
 './../../../data_new/raw/PnP\\PnP_7\\PnP_7_L.set',
 './../../../data_new/raw/PnP\\PnP_8\\PnP_8_L.set',
 './../../../data_new/raw/PnP\\PnP_9\\PnP_9_L.set']

In [13]:
pp_rh_raw = get_datasets(location_pain, rh)
pnp_rh_raw = get_datasets(location_nopain, rh)

Extracting parameters from ./../../../data/raw/PP\PP1\PP1_F1_RH_Removed_ICA.set...


<ipython-input-10-ca455454a3e4>:10: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  sets.append(mne.io.read_epochs_eeglab(path))


57 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from ./../../../data/raw/PP\PP10\PP10_F1_RH_Removed_ICA.set...


<ipython-input-10-ca455454a3e4>:10: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  sets.append(mne.io.read_epochs_eeglab(path))


57 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from ./../../../data/raw/PP\PP11\PP11_F1_RH_Removed_ICA.set...


...

58 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from ./../../../data/raw/PnP\PnP8\PnP8_F1_RH_Removed_ICA.set...


<ipython-input-10-ca455454a3e4>:10: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  sets.append(mne.io.read_epochs_eeglab(path))


53 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from ./../../../data/raw/PnP\PnP9\PnP9_F1_RH_Removed_ICA.set...
54 matching events found


<ipython-input-10-ca455454a3e4>:10: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  sets.append(mne.io.read_epochs_eeglab(path))


No baseline correction applied
Not setting metadata
0 projection items activated
Ready.


In [14]:
pp_rh_raw

array([array([[[ 1.44402540e-06,  9.81420040e-07,  6.11681223e-07, ...,
         -9.48653758e-07, -7.09739447e-07, -1.82451889e-07],
        [ 9.12692308e-07,  5.66146672e-07,  3.27923447e-07, ...,
         -3.50943863e-07, -1.84928596e-07,  2.25423664e-07],
        [ 3.08533460e-07, -1.32956609e-07, -4.29266930e-07, ...,
         -5.50954044e-07, -7.25167394e-07, -7.38074183e-07],
        ...,
        [-2.49450371e-07,  1.48201084e-06,  2.35830712e-06, ...,
          3.37429857e-06,  3.04905009e-06,  2.29389977e-06],
        [-5.30285418e-07,  8.94802094e-07,  1.52956212e-06, ...,
          3.27083945e-06,  2.76214886e-06,  1.80594766e-06],
        [-3.62152141e-09,  8.68661761e-07,  1.26055551e-06, ...,
          1.93071413e-06,  1.70576549e-06,  1.27043426e-06]],

       [[ 1.15336525e-06,  1.83704400e-06,  2.49183202e-06, ...,
         -9.63494658e-07, -7.82364130e-07, -3.78079295e-07],
        [ 3.36746752e-07,  6.24162018e-07,  9.46212053e-07, ...,
         -1.13700712e-06, -1.07

In [15]:
pp_rh_raw[4].shape

(51, 61, 1250)

In [16]:
pp_rh_bp = np.array([np.array([channels_bandpower(arr) for arr in patient]) for patient in pp_rh_raw])
pnp_rh_bp = np.array([np.array([channels_bandpower(arr) for arr in patient]) for patient in pnp_rh_raw])

In [17]:
pp_count = np.vstack(pp_rh_bp).shape[0]
pnp_count = np.vstack(pnp_rh_bp).shape[0]
pnp_count

473

In [18]:
pnp_rh_bp.shape

(9,)

Set some patients aside for testing

In [19]:
pp_and_pnp_bp = np.concatenate((pp_rh_bp, pnp_rh_bp))
pp_and_pnp_bp.shape

(19,)

In [20]:
# Returns a pair consisting of boolean (True is test patient is PP) and test label
def test_setup(test_index, total_size):
    test_is_pp = test_index < len(pp_rh_bp)
    test_label = 1 if test_is_pp else 0
    return test_is_pp, test_label

# Splits into train and test based on the index of the test patient
# Returns pair of test and train
def get_train_test(data, test_index):
    return data[test_index], np.delete(data, test_index)

# Returns pair of the lengths of PP train data and respectively PNP train data
def get_pp_pnp_length(pp_count, pnp_count, test_count, test_is_pp):
    pp_train_len = pp_count if not test_is_pp else pp_count - test_count
    pnp_train_len = pnp_count if test_is_pp else pnp_count - test_count
    return pp_train_len, pnp_train_len

# Ravel first dimention so that trials from all patients are treated separately; select channels
def ravel_all_trials(data, channels):
    return np.array(list(map(np.ravel, data[:, channels, :])))

In [21]:
test_index = 4

test_is_pp, test_label = test_setup(test_index, len(pp_rh_bp))
test_label

1

In [22]:
test_p, train_p = get_train_test(pp_and_pnp_bp, test_index)
test_p.shape

(51, 61, 3)

In [23]:
train_p_separated = np.vstack(train_p)
train_p_separated.shape

(952, 61, 3)

In [24]:
pp_train_len, pnp_train_len = get_pp_pnp_length(pp_count, pnp_count, len(test_p), test_is_pp)
pp_train_len

479

In [25]:
mul = 1

In [26]:
selected_channels = [10, 11]

In [27]:
train = ravel_all_trials(train_p_separated, selected_channels) * mul
train.shape

(952, 6)

In [28]:
test = ravel_all_trials(test_p, selected_channels) * mul
test.shape

(51, 6)

In [29]:
pca(train).shape

(952, 5)

In [30]:
labels = [1] * pp_train_len + [0] * pnp_train_len
test_labels = [test_label] * len(test)

It's time to learn

In [31]:
from sklearn import neighbors, svm
from sklearn.model_selection import train_test_split

In [32]:
knn = neighbors.KNeighborsClassifier(n_neighbors=130)

In [33]:
x_train, x_test, y_train, y_test = train_test_split(train, labels, test_size=0.05)

In [34]:
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=130, p=2,
                     weights='uniform')

In [35]:
knn.score(x_train, y_train)

0.7610619469026548

In [36]:
knn.score(x_test, y_test)

0.75

In [37]:
knn.predict(test)

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [38]:
np.count_nonzero(knn.predict(test) == test_labels)/len(test)

0.058823529411764705

### Cross validation

In [39]:
def classify_knn_with_xvalid(data_pp_bp, data_pnp_bp, n_neighbours, selected_channels, test_index, mul, verbose=True):
    
    data_bp = np.concatenate((data_pp_bp, data_pnp_bp))
    
    test_is_pp, test_label = test_setup(test_index, len(data_pp_bp))
    test_p, train_p = get_train_test(data_bp, test_index)
    train_p_separated = np.vstack(train_p)
    pp_train_len, pnp_train_len = get_pp_pnp_length(pp_count, pnp_count, len(test_p), test_is_pp)
    
    train = pca(ravel_all_trials(train_p_separated, selected_channels) * mul, n_components=3)
    test = pca(ravel_all_trials(test_p, selected_channels) * mul, n_components=3)
    
    labels = [1] * pp_train_len + [0] * pnp_train_len
    test_labels = [test_label] * len(test)
    
    if verbose:
        print('Test index', test_index, 'Testing on patient with', len(test), 'repetitions.')
    
    clas = neighbors.KNeighborsClassifier(n_neighbors=n_neighbours)
    clas.fit(train, labels)
    train_acc = clas.score(train, labels)
    test_acc = clas.score(test, test_labels)
    
    if verbose:
        print('Train score:', train_acc, '  Test score:', test_acc)
    
    return test_acc
    
    

In [40]:
classify_knn_with_xvalid(pp_rh_bp, pnp_rh_bp, 23, [0, 3, 10, 36], 2, 10000000000000)

Test index 2 Testing on patient with 59 repetitions.
Train score: 0.8167372881355932   Test score: 0.559322033898305


0.559322033898305

#### Cross validate over the whole dataset

In [41]:
total_score = 0
patients_correct = 0
for i in range(len(pp_and_pnp_bp)):
    score = classify_knn_with_xvalid(pp_rh_bp, pnp_rh_bp, 9, [5, 16, 24, 36, 60], i, mul)
    total_score += score
    if score > 0.5:
        patients_correct += 1
    
# TODO log acc for each patient
    
print('Overall accuracy', total_score/len(pp_and_pnp_bp))
print('Correctly labeled', patients_correct, 'out of', len(pp_and_pnp_bp))

Test index 0 Testing on patient with 57 repetitions.
Train score: 0.872093023255814   Test score: 0.8947368421052632
Test index 1 Testing on patient with 57 repetitions.
Train score: 0.864693446088795   Test score: 0.9298245614035088
Test index 2 Testing on patient with 59 repetitions.
Train score: 0.888771186440678   Test score: 0.5084745762711864
Test index 3 Testing on patient with 54 repetitions.
Train score: 0.863013698630137   Test score: 0.5925925925925926
Test index 4 Testing on patient with 51 repetitions.
Train score: 0.8560924369747899   Test score: 1.0
Test index 5 Testing on patient with 58 repetitions.
Train score: 0.8634920634920635   Test score: 0.896551724137931
Test index 6 Testing on patient with 56 repetitions.
Train score: 0.8500527983104541   Test score: 1.0
Test index 7 Testing on patient with 32 repetitions.
Train score: 0.8558187435633368   Test score: 1.0
Test index 8 Testing on patient with 52 repetitions.
Train score: 0.8527865404837014   Test score: 0.90384

#### Cross validate over multiple channels

In [100]:
file = open('all_results/bandpower_pca_knn_results.csv', 'a', newline='')
n_components = 3
name = 'Bandpower + PCA + KNN'
notes = 'freq bands 4-8,8-13,13-30, k='
notes_c = ', n_components='

previous_channels = [0, 2, 4]
k = 19

max_acc = {'index': 0, 'value': 0}
for channel in range(61):    
    total_score = 0
    correct_patients = 0
    for i in range(len(pp_and_pnp_bp)):
        score = classify_knn_with_xvalid(pp_rh_bp, pnp_rh_bp, k, previous_channels + [channel], i, mul, verbose=False)
        total_score += score
        if score > 0.5:
            correct_patients += 1
        
    avg_score = total_score/len(pp_and_pnp_bp)
    print(channel, avg_score, correct_patients)
    
    log_result(file, name, avg_score, correct_patients, len(pp_and_pnp_bp), 'RH', str(previous_channels + [channel]), notes + str(k) + notes_c + str(n_components))
    
    if avg_score > max_acc['value']:
        max_acc['index'] = channel
        max_acc['value'] = avg_score
        
file.close()
        
print('Max accuracy:', max_acc['index'], max_acc['value'])

SyntaxError: invalid syntax (<ipython-input-100-904480d10a0a>, line 23)

### Cross validate over multiple n_neighbours

In [93]:
file = open('all_results/bandpower_pca_knn_results.csv', 'a', newline='')
n_components = 3
name = 'Bandpower + PCA + KNN'
notes = 'freq bands 4-8,8-13,13-30, k='
notes_c = ', n_components='

channels = [0, 2, 4]

max_acc = {'index': 0, 'value': 0}
for n_neighbours in range(11, 31, 1):    
    total_score = 0
    correct_patients = 0
    for i in range(len(pp_and_pnp_bp)):
        score = classify_knn_with_xvalid(pp_rh_bp, pnp_rh_bp, n_neighbours, channels, i, mul, verbose=False)
        total_score += score
        if score > 0.5:
            correct_patients += 1
        
    avg_score = total_score/len(pp_and_pnp_bp)
    print(n_neighbours, avg_score, correct_patients)
    
    log_result(file, name, avg_score, correct_patients, len(pp_and_pnp_bp), 'RH', str(channels), notes + str(n_neighbours) + notes_c + str(n_components))
    
    if avg_score > max_acc['value']:
        max_acc['index'] = n_neighbours
        max_acc['value'] = avg_score
        
print('Max accuracy:', max_acc['index'], max_acc['value'])

11 0.5398321193631466 9
12 0.5340091590947704 10
13 0.5410384474147698 9
14 0.5453917799224375 9
15 0.5523969920680644 9
16 0.5490582640076835 9
17 0.5526308595478538 10
18 0.5514737455150122 9
19 0.5645185917208824 10
20 0.5502381991441528 8
21 0.5489941026958316 9
22 0.5480797716916167 8
23 0.5444912540343368 9
24 0.5473287755319026 8
25 0.5463002081750449 9
26 0.547152106473844 8
27 0.5471493554059607 9
28 0.5438752210167402 8
29 0.5443109173062749 9
30 0.5369980482365749 8
Max accuracy: 19 0.5645185917208824


### SVM

In [42]:
import warnings
warnings.filterwarnings('ignore')

In [43]:
def classify_nusvm_with_xvalid(data_pp_bp, data_pnp_bp, nu, selected_channels, test_index, mul, verbose=True):
    
    data_bp = np.concatenate((data_pp_bp, data_pnp_bp))
    
    test_is_pp, test_label = test_setup(test_index, len(data_pp_bp))
    test_p, train_p = get_train_test(data_bp, test_index)
    train_p_separated = np.vstack(train_p)
    pp_train_len, pnp_train_len = get_pp_pnp_length(pp_count, pnp_count, len(test_p), test_is_pp)
    
    train = pca(ravel_all_trials(train_p_separated, selected_channels) * mul, n_components=3)
    test = pca(ravel_all_trials(test_p, selected_channels) * mul, n_components=3)
    
    labels = [1] * pp_train_len + [0] * pnp_train_len
    test_labels = [test_label] * len(test)
    
    if verbose:
        print('Test index', test_index, 'Preparing to classify set of', pp_train_len, 'PP and', pnp_train_len, 'PNP.')
    
    clas = svm.NuSVC(nu=nu, kernel='linear')
    clas.fit(train, labels)
    train_acc = clas.score(train, labels)
    test_acc = clas.score(test, test_labels)
    
    if verbose:
        print('Train score:', train_acc, '  Test score:', test_acc)
    
    return test_acc
    

In [61]:
total_score = 0
patients_correct = 0
for i in range(len(pp_and_pnp_bp)):
    score = classify_nusvm_with_xvalid(pp_rh_bp, pnp_rh_bp, 0.8585, [0, 1, 1, 2, 3, 3, 5, 12, 13, 23, 30, 52, 57], i, mul)
    total_score += score
    if score > 0.5:
        patients_correct += 1
    

print(total_score/len(pp_and_pnp_bp))
print('Correctly labeled', patients_correct, 'out of', len(pp_and_pnp_bp))

Test index 0 Preparing to classify set of 473 PP and 473 PNP.
Train score: 0.6807610993657506   Test score: 0.9649122807017544
Test index 1 Preparing to classify set of 473 PP and 473 PNP.
Train score: 0.7071881606765328   Test score: 1.0
Test index 2 Preparing to classify set of 471 PP and 473 PNP.
Train score: 0.6917372881355932   Test score: 0.5254237288135594
Test index 3 Preparing to classify set of 476 PP and 473 PNP.
Train score: 0.6891464699683878   Test score: 0.9814814814814815
Test index 4 Preparing to classify set of 479 PP and 473 PNP.
Train score: 0.7121848739495799   Test score: 1.0
Test index 5 Preparing to classify set of 472 PP and 473 PNP.
Train score: 0.6835978835978836   Test score: 0.9827586206896551
Test index 6 Preparing to classify set of 474 PP and 473 PNP.
Train score: 0.7286166842661035   Test score: 1.0
Test index 7 Preparing to classify set of 498 PP and 473 PNP.
Train score: 0.713697219361483   Test score: 1.0
Test index 8 Preparing to classify set of 478

In [60]:
file = open('all_results/bandpower_pca_svm_results.csv', 'a', newline='')
n_components = 3
name = 'Bandpower + PCA + SVM'
notes = 'freq bands 4-8,8-13,13-30, nu='
notes_c = ', n_components='

previous_channels=[0, 1, 1, 2, 3, 3, 5, 12, 13, 23, 30, 52, 57]
nu = 0.8585

max_acc = {'index': 0, 'value': 0}
for channel in range(61):    
    total_score = 0
    correct_patients = 0
    for i in range(len(pp_and_pnp_bp)):
        score = classify_nusvm_with_xvalid(pp_rh_bp, pnp_rh_bp, nu, previous_channels + [channel], i, mul, verbose=False)
        total_score += score
        if score > 0.5:
            correct_patients += 1
        
    avg_score = total_score/len(pp_and_pnp_bp)
    print(channel, avg_score, correct_patients)
    
    log_result(file, name, avg_score, correct_patients, len(pp_and_pnp_bp), 'RH', str(previous_channels + [channel]), notes + str(nu) + notes_c + str(n_components))
    
    if avg_score > max_acc['value']:
        max_acc['index'] = channel
        max_acc['value'] = avg_score
        

file.close()
        
print('Max accuracy:', max_acc['index'], max_acc['value'])

0 0.9284084893090333 18
1 0.9284084893090333 18
2 0.929383148178429 18
3 0.9586901321133621 19
4 0.9577667710791978 19
5 0.9596647909827578 19
6 0.9577154732439666 19
7 0.9257169269724497 17
8 0.9099571886435767 18
9 0.9265936072763654 18
10 0.9265936072763654 18
11 0.9284910875183041 18
12 0.9605722319990919 19
13 0.9577980714532375 19
14 0.9069659941823278 18
15 0.9586229142603004 19
16 0.956673596521509 19
17 0.9479688845500093 19
18 0.9275164286489086 18
19 0.9275164286489086 18
20 0.9069659941823278 18
21 0.9069506138261187 18
22 0.9051511121496599 18
23 0.9033823711856193 17
24 0.923850207442929 17
25 0.9257841448255116 17
26 0.911989104591639 18
27 0.9257015466162408 18
28 0.9265936072763654 18
29 0.9285583053713661 18
30 0.9577826910970283 19
31 0.9051664925058691 18
32 0.9041764532802642 18
33 0.9070332120353894 18
34 0.9040420175741406 18
35 0.9059913353129321 18
36 0.9032843926201393 18
37 0.9041246157834114 18
38 0.8968286305120964 17
39 0.9024077123162235 17
40 0.891264389

### Cross validate over multiple nu values

In [55]:
file = open('all_results/bandpower_pca_svm_results.csv', 'a', newline='')
n_components = 3
name = 'Bandpower + PCA + SVM'
notes = 'freq bands 4-8,8-13,13-30, nu='
notes_c = ', n_components='

channels = [0, 1, 1, 2, 5, 12, 13, 23, 30, 52, 57]

max_acc = {'index': 0, 'value': 0}
for param in np.arange(0.1, 0.875, 0.0005):    
    total_score = 0
    correct_patients = 0
    for i in range(len(pp_and_pnp_bp)):
        score = classify_nusvm_with_xvalid(pp_rh_bp, pnp_rh_bp, param, channels, i, mul, verbose=False)
        total_score += score
        if score > 0.5:
            correct_patients += 1
        
    avg_score = total_score/len(pp_and_pnp_bp)
    print(param, avg_score, correct_patients)
    
    log_result(file, name, avg_score, correct_patients, len(pp_and_pnp_bp), 'RH', str(channels), notes + str(nu) + notes_c + str(n_components))
    
    if avg_score > max_acc['value']:
        max_acc['index'] = param
        max_acc['value'] = avg_score
        
print('Max accuracy:', max_acc['index'], max_acc['value'])

0.1 0.4160451264393654 8
0.1005 0.41795900203745157 8
0.101 0.42527221514013835 8
0.1015 0.42769948010555076 8
0.10200000000000001 0.42045533233108817 8
0.10250000000000001 0.42052255018414997 8
0.10300000000000001 0.42368044492099205 8
0.10350000000000001 0.42164438720626396 8
0.10400000000000001 0.4346192017380563 9
0.10450000000000001 0.4345519838849945 9
0.10500000000000001 0.4267549573583855 9
0.10550000000000001 0.43445690117648167 9
0.10600000000000001 0.4629194345987752 9
0.10650000000000001 0.46384459668546174 9
0.10700000000000001 0.46478333807583516 9
0.10750000000000001 0.42739298719864216 8
0.10800000000000001 0.4274937818165827 8
0.10850000000000001 0.46255410647363443 9
0.10900000000000001 0.4625718275439872 9
0.10950000000000001 0.5287647651074048 11
0.11000000000000001 0.5650596708496568 11
0.11050000000000001 0.5739778998028021 11
0.11100000000000002 0.5400072408321431 10
0.11150000000000002 0.5417811056199906 10
0.11200000000000002 0.44034240584464734 8
0.11250000000

0.2000000000000001 0.5076816334621865 9
0.2005000000000001 0.517659497142102 10
0.2010000000000001 0.517659497142102 10
0.2015000000000001 0.5676148941090957 11
0.2020000000000001 0.5676148941090957 11
0.2025000000000001 0.5705388707172828 11
0.2030000000000001 0.5705388707172828 11
0.2035000000000001 0.522413736139166 10
0.2040000000000001 0.522413736139166 10
0.2045000000000001 0.5214390772697703 10
0.2050000000000001 0.5214390772697703 10
0.2055000000000001 0.5205316362534363 10
0.2060000000000001 0.5620659805888512 11
0.2065000000000001 0.5631186121677986 11
0.2070000000000001 0.5592199766902157 11
0.2075000000000001 0.6100366736049162 12
0.2080000000000001 0.6147289126771732 11
0.2085000000000001 0.6147289126771732 11
0.2090000000000001 0.5669706280767833 10
0.2095000000000001 0.5669706280767833 10
0.2100000000000001 0.46782208459651625 9
0.2105000000000001 0.5721105836218574 11
0.2110000000000001 0.4288357298206878 9
0.2115000000000001 0.4814673087680562 10
0.2120000000000001 0.4

0.2995000000000002 0.5940184985210573 11
0.30000000000000016 0.5940184985210573 11
0.3005000000000002 0.5820461054509601 11
0.30100000000000016 0.590966712052209 11
0.3015000000000002 0.5958400063991875 11
0.30200000000000016 0.5948830686001445 11
0.3025000000000002 0.5428055062732747 9
0.30300000000000016 0.5406448909996392 9
0.3035000000000002 0.5523407974323876 9
0.30400000000000016 0.5485130462362156 9
0.3045000000000002 0.6135176630413862 11
0.30500000000000016 0.6158013211646942 11
0.3055000000000002 0.616693381824819 11
0.30600000000000016 0.6058835289097031 11
0.3065000000000002 0.6058835289097031 11
0.30700000000000016 0.5838343042145714 11
0.3075000000000002 0.586510486194946 11
0.30800000000000016 0.5855358273255503 11
0.3085000000000002 0.5855358273255503 11
0.30900000000000016 0.5855517473433808 11
0.3095000000000002 0.5828755653630061 11
0.31000000000000016 0.586774200840589 11
0.3105000000000002 0.5858172630415459 11
0.31100000000000017 0.5959742344173538 11
0.3115000000

0.40000000000000024 0.4662065551254964 10
0.4005000000000003 0.3912708586643821 9
0.40100000000000025 0.4376823419179705 10
0.4015000000000003 0.3916297103390233 9
0.40200000000000025 0.26244795027083656 6
0.4025000000000003 0.31870929328354075 7
0.40300000000000025 0.41359282980846884 8
0.4035000000000003 0.3774731187661572 7
0.40400000000000025 0.39855061006525516 7
0.4045000000000003 0.4248663995389394 7
0.40500000000000025 0.418617349418346 7
0.4055000000000003 0.41956959634582514 7
0.40600000000000025 0.3938730802580348 7
0.4065000000000003 0.405669813470376 8
0.40700000000000025 0.4085111918692923 7
0.4075000000000003 0.4027695650750339 7
0.40800000000000025 0.5047625486748418 10
0.4085000000000003 0.42755046219730497 8
0.40900000000000025 0.4737641602843304 10
0.4095000000000003 0.3823766004757179 8
0.41000000000000025 0.3922450215283495 8
0.4105000000000003 0.39123287577936167 8
0.41100000000000025 0.28007135127845423 6
0.4115000000000003 0.3074397723310858 7
0.4120000000000002

0.5000000000000003 0.5060404840956623 10
0.5005000000000004 0.5060404840956623 10
0.5010000000000003 0.5586720630430307 11
0.5015000000000004 0.5023470399590049 10
0.5020000000000003 0.47699515449822655 10
0.5025000000000004 0.4145509082894843 8
0.5030000000000003 0.46718248723685274 9
0.5035000000000004 0.4708759313735102 9
0.5040000000000003 0.4647254978183579 9
0.5045000000000004 0.46829374045885747 9
0.5050000000000003 0.41566216151148905 8
0.5055000000000004 0.40919863427233855 7
0.5060000000000003 0.44005162882769244 7
0.5065000000000004 0.4980355717358102 9
0.5070000000000003 0.4980355717358102 9
0.5075000000000004 0.4454039927884418 8
0.5080000000000003 0.4234909726903038 8
0.5085000000000004 0.3708593937429353 7
0.5090000000000003 0.4234909726903038 8
0.5095000000000004 0.4234909726903038 8
0.5100000000000003 0.4788926347401652 10
0.5105000000000004 0.44434265826641384 9
0.5110000000000003 0.3863587153582961 7
0.5115000000000004 0.3863587153582961 7
0.5120000000000003 0.390052

0.6010000000000004 0.4922120401156976 10
0.6015000000000005 0.4338388343740707 9
0.6020000000000004 0.4329467737139458 9
0.6025000000000004 0.4329467737139458 9
0.6030000000000004 0.4348960914527373 9
0.6035000000000005 0.4875276704001057 10
0.6040000000000004 0.4875276704001057 10
0.6045000000000004 0.4959487230316847 10
0.6050000000000004 0.4959487230316847 10
0.6055000000000005 0.4959487230316847 10
0.6060000000000004 0.4959487230316847 10
0.6065000000000004 0.4959487230316847 10
0.6070000000000004 0.43107542868470605 9
0.6075000000000005 0.43107542868470605 9
0.6080000000000004 0.43681705547896443 9
0.6085000000000004 0.43681705547896443 9
0.6090000000000004 0.4357644239000171 9
0.6095000000000005 0.48742134397798975 10
0.6100000000000004 0.39546560235119554 8
0.6105000000000004 0.3892211777303214 7
0.6110000000000004 0.39448433562505825 7
0.6115000000000005 0.3993576299720368 7
0.6120000000000004 0.42307174480457266 8
0.6125000000000004 0.4293161694254468 9
0.6130000000000004 0.42

0.7020000000000005 0.37791684312647733 8
0.7025000000000005 0.3769046973774895 8
0.7030000000000005 0.3262364908109163 7
0.7035000000000006 0.3841312276530216 8
0.7040000000000005 0.3841312276530216 8
0.7045000000000005 0.4358394455662257 9
0.7050000000000005 0.382134801752689 8
0.7055000000000006 0.3716084859632153 8
0.7060000000000005 0.366256122002466 7
0.7065000000000005 0.31854913523730743 6
0.7070000000000005 0.23063269595970662 4
0.7075000000000006 0.283264274907075 5
0.7080000000000005 0.3358958538544434 6
0.7085000000000005 0.31142678644908794 5
0.7090000000000005 0.3497042984108104 6
0.7095000000000006 0.297072719463442 5
0.7100000000000005 0.25068556513694784 5
0.7105000000000005 0.17368751445468664 4
0.7110000000000005 0.22631909340205505 5
0.7115000000000006 0.27895067234942345 6
0.7120000000000005 0.2727062477285493 5
0.7125000000000005 0.3769947467538904 7
0.7130000000000005 0.32436316780652197 6
0.7135000000000006 0.27173158885915355 5
0.7140000000000005 0.2717315888591

0.8030000000000006 0.7915402572279129 14
0.8035000000000007 0.8423251141069527 15
0.8040000000000006 0.8760093246332685 16
0.8045000000000005 0.7698387257221977 14
0.8050000000000006 0.7698387257221977 14
0.8055000000000007 0.7435229362485135 14
0.8060000000000006 0.6382597783537767 12
0.8065000000000005 0.7408006131995116 14
0.8070000000000006 0.7408006131995116 14
0.8075000000000007 0.7954641080949423 15
0.8080000000000006 0.7954641080949423 15
0.8085000000000006 0.8517254511076465 16
0.8090000000000006 0.8517254511076465 16
0.8095000000000007 0.8430481337773408 16
0.8100000000000006 0.8439714948115051 16
0.8105000000000006 0.8439714948115051 16
0.8110000000000006 0.8448789358278391 16
0.8115000000000007 0.855175178059853 17
0.8120000000000006 0.855175178059853 17
0.8125000000000006 0.749912020165116 15
0.8130000000000006 0.8034510401288184 16
0.8135000000000007 0.8054003578676098 16
0.8140000000000006 0.8063237189017741 16
0.8145000000000006 0.8063237189017741 16
0.8150000000000006 

In [112]:
file.close()